# EfficientNet Model Repurposing - Experiments

In [1]:
%pip install lightning

  Using cached lightning-2.5.1.post0-py3-none-any.whl.metadata (39 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached pytorch_lightning-2.5.1.post0-py3-none-any.whl.metadata (20 kB)
  Using cached aiohttp-3.11.18-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.6.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached multidict-6.4.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached yarl-1.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metada

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import models
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


In [3]:
data_dir = r"/exchange/dspro2/silent-speech/ASL_Pictures_Dataset"
batch_size = 32 * 2 ** 3
img_size = 224
num_workers = 64

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
train_dataset = datasets.ImageFolder(root=f"{data_dir}/Train", transform=data_transforms)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/Test", transform=data_transforms)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [7]:
class_names = train_dataset.classes
num_classes = len(class_names)

In [8]:
class ASLCNN(pl.LightningModule):
    def __init__(self, kernel_size=3, dropout=0.2, hidden_dim=128, num_classes=28):
        super().__init__()
        self.save_hyperparameters()
        
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.hidden_dim = hidden_dim
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=self.kernel_size, stride=1, padding=1), # 400 -> 200
            nn.ReLU(),
            nn.MaxPool2d(2),  # 200 -> 100
            
            nn.Conv2d(32, 64, kernel_size=self.kernel_size, padding=1),  # 100
            nn.ReLU(),
            nn.MaxPool2d(2),  # 100 -> 50
            
            nn.Conv2d(64, 128, kernel_size=self.kernel_size, padding=1), # 50
            nn.ReLU(),
            nn.MaxPool2d(2),  # 50 -> 25
        
            nn.Flatten(),
            nn.LazyLinear(self.hidden_dim), #nn.Linear(128 * 25 * 25, 512),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dim, num_classes),  # number of classes
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("train_loss", loss, prob_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("val_los", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)



In [9]:
# Initialize model
model = ASLCNN()


In [10]:
# Define callbacks
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, filename="CNN_best_model")

In [11]:
# Initialize Trainer
trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [12]:
# Train model
trainer.fit(model, train_loader, val_loader)

You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.12/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:477: The total number of parameters detected may be inaccurate because the model contains an instance of `UninitializedParameter`. To get an accurate number, set `self.example_input_array` in your LightningModule.

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 96.9 K | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
96.9 K    Trainable params
0   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

NameError: name 'ouputs' is not defined

In [ ]:
# Save the final model
torch.save(model.state_dict(), "asl_cnn_model.pth")